In [12]:
import json

# Open the JSON file
with open('medals.json') as f:
    # Load the JSON data from the file
    data = json.load(f)

# Access the data
medals = [(medal['Id'],medal['Name'],medal['Description'],medal['Emote'],medal.get('Role'),medal['created']) for medal in data]
for medal in medals:
    print(medal)

(1, 'Frostarian War Commendation - LSE War', 'War commendation given to medal recipients for the LSE war', '🎖️', None, 1612824854932)
(2, '2021 Frostarian Easter Egg Hunt', 'Found all the eggs during the Frostarian 2021 Egg Hunt at Grand Celeste', '🥚', None, 1617320834150)
(3, 'Voice of the Vanguard - Snow Core War', "Hosted the most raid wins on Aurora's Dam during the Snow Core war", '❄️', None, 1623373111708)
(4, 'Bulwark of Celeste - Snow Core War', 'Hosted the most defense wins at Celeste 2 during the Snow Core wwar', '❄️', None, 1623373123713)
(5, 'Invader Inquisitor - Snow Core War', "Attended the most Aurora's Dam raids during the Snow Core war", '❄️', None, 1623373135931)
(6, 'Celeste Protector - Snow Core War', 'Attended the most Celeste 2 defenses during the Snow Core war', '❄️', None, 1623373148410)
(7, 'Frequent Flyer - Snow Core War', 'Attended the most events overall throughout the Snow Core war', '❄️', None, 1623373156626)
(8, "Soldier's Savior - Snow Core War", 'Accumu

In [8]:
import os,requests

def get_usernames_from_ids(ids):
    try:
        url = 'https://users.roblox.com/v1/users'
        request = requests.post(url, json={
            'userIds': ids,
            'excludeBannedUsers': True
        })
        response = request.json()['data']
        return [{'username': user.get('name'), 'id': user.get('id')} for user in response]
    except:
        return [] 
    
def get_roblox_ids(usernames):
    while "  " in usernames:
        usernames = usernames.replace("  ", " ")
    usernames = usernames.split(" ")

    # First, process the usernames that are already numeric strings
    ids = [id for id in usernames if id.isdigit()]
    results = get_usernames_from_ids(ids)
    
    # Next, process the usernames that are Discord mentions
    for username in usernames:
        if username.startswith('<@') and username.endswith('>'):
            # user rover to get their roblox id
            API = os.getenv('ROVERIFY_API')
            GUILD_ID = os.getenv('DISCORD_GUILD')
            url = f"http://registry.rover.link/api/guilds/{GUILD_ID}/discord-to-roblox/{username[2:-1]}"
            request = requests.get(url,headers={'Authorization': f"Bearer {API}"})
            response = request.json()

            if response.get('robloxId'):
                results.append({'username':response.get('cachedUsername'), 'id':response.get('robloxId')})
            else:
                results.append({'username':username, 'id':None})
    
    
    remaining_usernames = [u for u in usernames if u not in results and not username.startswith('<@')]
    if remaining_usernames:
        try:
            url = 'https://users.roblox.com/v1/usernames/users'
            request = requests.post(url, json={
                'usernames': remaining_usernames,
                'excludeBannedUsers': True
            })
            response = request.json()['data']
            
            # May not include all ids if a user with this name on roblox does not exist
            for user in response:
                results.append({'username':user.get('name'), 'id':user.get('id')})

            for user in remaining_usernames:
                if user not in [user['username'] for user in results]:
                    results.append({'username':user, 'id':None})
        except:
            pass

    # lastly remove any duplicates
    return [dict(t) for t in {tuple(d.items()) for d in results}]


In [31]:
import pandas as pd
df = pd.read_csv('../../medals.csv')
df

from sql import create_db_connection, execute_query, read_query
# from lib.roblox.roblox_functions import get_roblox_ids
import os

SQL_PASS = os.getenv('SQL_PASSWORD')
db = create_db_connection('localhost','root',SQL_PASS,'guf')

MySQL Database connection successful


In [32]:
name = df.columns[0]
print(name)
get_medal_query = f"""
SELECT Medal_ID FROM medals WHERE Name = '{name}';
"""
medal_id = read_query(db,get_medal_query)
print(medal_id)
for col in df.columns:
    print(col)

Frostarian War Commendation - LSE War
[(1,)]
Frostarian War Commendation - LSE War
2021 Frostarian Easter Egg Hunt
Voice of the Vanguard - Snow Core War
Bulwark of Celeste - Snow Core War
Invader Inquisitor - Snow Core War
Celeste Protector - Snow Core War
Frequent Flyer - Snow Core War
Soldier's Savior - Snow Core War
Into the Breach - Snow Core War
May 2021 Frostarian Video Montage
Best Hoster 2021
Best Aimer 2021
Best Defender 2021
Most likely to throw a defense 2021
Cult Leader 2021
Funniest Frostarian 2021
Frostarian Legend 2021
Most Active 2021
Most Productive Officer 2021
Most Epik 2021
Most likely to rage quit 2021
Most likely to get moderated 2021
Loudest on the Mic 2021
Worst Aimer 2021
The Stinkiest Poop Man 2021
Most Toxic 2021
Most likely to be demoted 2021
Best Dresser 2021
Worst Dresser 2021
The Ultimate Femboy 2021
Biggest Weeb 2021
Doomspire Tournament Winner Summer 2021
Developer Award
Top Moderator
Frostarian Times Pulitzer
Most Nominations
Voice of the Vanguard - WA

In [36]:
for medal in df.columns:
    print(medal)
    for player in df[medal]:
        if type(player) != str:
            continue
        id = get_roblox_ids(player)[0]['id']
        try:
            query = f"""
            INSERT INTO users VALUES ({id},0,0,0,0,0);
            """
            execute_query(db, query)
        except:
            print(f"{player} already added to the database")

        get_medal_query = f"""
        SELECT Medal_ID FROM medals WHERE Name = "{medal}";
        """
        medal_id = read_query(db,get_medal_query)[0][0]

        try:
            query = f"""
            INSERT INTO user_medals VALUES ({id},{medal_id});
            """
            execute_query(db, query)
        except:
            print("Insert medal went wrong")


    print()

Frostarian War Commendation - LSE War
error with 1062 (23000): Duplicate entry '11752468' for key 'users.PRIMARY'
Shinyehh already added to the database
error with 1062 (23000): Duplicate entry '11752468-1' for key 'user_medals.PRIMARY'
Insert medal went wrong
error with 1062 (23000): Duplicate entry '4460325' for key 'users.PRIMARY'
runesman181 already added to the database
error with 1062 (23000): Duplicate entry '4460325-1' for key 'user_medals.PRIMARY'
Insert medal went wrong

2021 Frostarian Easter Egg Hunt
error with 1062 (23000): Duplicate entry '11752468' for key 'users.PRIMARY'
Shinyehh already added to the database
error with 1062 (23000): Duplicate entry '11752468-2' for key 'user_medals.PRIMARY'
Insert medal went wrong
error with 1062 (23000): Duplicate entry '13911940' for key 'users.PRIMARY'
iSython already added to the database
error with 1062 (23000): Duplicate entry '13911940-2' for key 'user_medals.PRIMARY'
Insert medal went wrong
error with 1062 (23000): Duplicate en